In [9]:
from sqlalchemy import create_engine, Column, Date, String, Integer, Float, exc, and_, or_
from sqlalchemy.orm import Session, declarative_base

import requests

from selectorlib import Extractor
import requests
import json
from time import sleep
import random
import re

In [10]:
def scrape(url, file):
    e = Extractor.from_yaml_file(file)
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    # Download the page using requests
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        return None

    # Pass the HTML of the page and create
    return e.extract(r.text)

In [11]:
engine = create_engine("sqlite+pysqlite:///books.sqlite")
Base = declarative_base()
Base.metadata.create_all(engine)
session = Session(engine)

In [12]:
class Amazon(Base):
    __tablename__ = "Amazon"

    isbn = Column(String, primary_key=True, nullable=False)
    title = Column(String, nullable=True)
    subtitle = Column(String, nullable=True)
    author = Column(String, nullable=True)
    description = Column(String, nullable=True)
    book_type = Column(String, nullable = True)
    price = Column(Float, nullable=True)
    rating = Column(Float, nullable=True)
    review_count = Column(Integer, nullable=True)
    page_count = Column(Integer, nullable=True)
    language = Column(String, nullable=True)
    publisher = Column(String, nullable=True)
    published_date = Column(String, nullable=True)
    url = Column(String, nullable=True)
    

    def __repr__(self) -> str:
        return (
            f"Amazon(isbn={self.isbn!r}, "
            f"title={self.title!r}, "
            f"subtitle={self.subtitle!r}, "
            f"author={self.author!r}, "
            f"description={self.description!r}, "
            f"book_type={self.book_type!r}, "
            f"price={self.price!r}, "
            f"rating={self.rating!r}, "
            f"review_count={self.review_count!r}, "
            f"page_count={self.page_count!r}, "
            f"language={self.language!r}, "
            f"publisher={self.publisher!r}, "
            f"published_date={self.published_date!r}, "
            f"url={self.url!r})"
        )


In [13]:
def scrapeAmazon(level):
    books = (
        session.query(Amazon).filter(((Amazon.title == None) | (Amazon.title == -level)) & (Amazon.rating != None)).all()
    )
    print(len(books))
    for book in books:
        try:
            amazon = scrape(book.url, "definitions/amazon.yml")
            print(amazon, book.url)
            
            
            if amazon != None:
                changed_flag = False
                

                if amazon["title"] != None:
                    book.title = amazon["title"]
                    changed_flag = True
                
                if amazon["subtitle"] != None:
                    book.subtitle = amazon["subtitle"]
                    changed_flag = True

                if amazon["author"] != None:
                    book.author = amazon["author"]
                    changed_flag = True

                if amazon["description"] != None:
                    book.description = amazon["description"]
                    changed_flag = True

                if amazon["price"] != None:
                    book.price = re.sub(r'^.*?\$', '', amazon["price"])
                    changed_flag = True
                
                if amazon["page_count"] != None:
                    book.page_count = amazon["page_count"]
                    changed_flag = True
                
                if amazon["language"] != None:
                    book.language = amazon["language"]
                    changed_flag = True

                if amazon["publisher"] != None:
                    book.publisher = amazon["publisher"]
                    changed_flag = True

                if amazon["published_date"] != None:
                    book.published_date = amazon["published_date"]
                    changed_flag = True

                if amazon["book_type"] != None:
                    book.book_type = amazon["book_type"]
                    changed_flag = True


                if not changed_flag:
                    try:
                        book.title = book.title - 1
                    except: # does the same thing, but seperated for semantic reasons
                        book.title = level - 1
                        
                session.commit()
        except:
            pass
        
        
        

In [19]:
#start i at 10
for i in range(20):
    
        scrapeAmazon(i+1)
    
        print("FAILED")

1227
{'title': None, 'subtitle': None, 'author': None, 'description': None, 'price': None, 'rating': None, 'review_count': None, 'page_count': None, 'language': None, 'publisher': None, 'published_date': None, 'book_type': None} https://www.amazon.com/Good-Omens/dp/B002WEBBBO?tag=NYTBSREV-20
{'title': None, 'subtitle': None, 'author': None, 'description': None, 'price': None, 'rating': None, 'review_count': None, 'page_count': None, 'language': None, 'publisher': None, 'published_date': None, 'book_type': None} https://www.amazon.com/Clancy-Enemy-Contact-Jack-Novel/dp/0525541691?tag=NYTBSREV-20
{'title': None, 'subtitle': None, 'author': None, 'description': None, 'price': None, 'rating': None, 'review_count': None, 'page_count': None, 'language': None, 'publisher': None, 'published_date': None, 'book_type': None} https://www.amazon.com/Oracle-Sam-Remi-Fargo-Adventure/dp/0525539611?tag=NYTBSREV-20
{'title': None, 'subtitle': None, 'author': None, 'description': None, 'price': None, 'ra

ConnectionError: HTTPSConnectionPool(host='www.amazon.com', port=443): Max retries exceeded with url: /Rebel-Heir-Book-One-Rush-ebook/dp/B07C1BQPN2?tag=NYTBSREV-20 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000018182537700>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [36]:
books = session.query(Amazon).filter(((Amazon.rating == None) | (Amazon.rating <= -2))  & (Amazon.url != None)).all()

In [18]:
session.rollback()

In [18]:
amazon = scrape("https://www.amazon.com/dp/1984818503?tag=NYTBSREV-20", "definitions/amazon.yml")

In [19]:
amazon


{'title': 'Better Off Dead A Jack Reacher Novel',
 'subtitle': 'Hardcover – October 26, 2021',
 'author': 'Lee Child',
 'description': "#1 NEW YORK TIMES BESTSELLER • Jack Reacher is back in a brand-new page-turning thriller from acclaimed #1 bestselling authors Lee Child and Andrew Child. Digging graves had not been part of my plans when I woke up that morning. Reacher goes where he wants, when he wants.\xa0That morning he was heading west, walking under the merciless desert sun—until he comes upon a curious scene.\xa0A Jeep has crashed into the only tree for miles around. A woman is slumped over the wheel. Dead?\xa0No, nothing is what it seems. The woman is Michaela Fenton, an army veteran turned FBI agent trying to find her twin brother, who might\xa0be mixed up with some dangerous people.\xa0Most of them would rather die than betray their terrifying leader, who has burrowed his influence deep into the nearby border town, a backwater that has seen better days.\xa0The mysterious Dend